## Import libraries 

In [0]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import pprint
import keras

## Data generator 

Keras does not have a generator that reads h5.
Pytorch handles h5 files.

In [0]:
import h5py

import sklearn.feature_extraction.image as ski

def create_patches(x1,params,bs):
  
    x2 = np.zeros((bs,params.psize[0],params.psize[1],params.nch),dtype='float')

    for i1 in np.arange(bs):
        x2[i1,:,:,0] = ski.extract_patches_2d(x1[i1,:,:,:],(params.psize[0],params.psize[1]),max_patches=params.npt)

        x2[i1,:,:,0] -= np.min(x2[i1,:,:,0])
        x2[i1,:,:,0] /= np.max(x2[i1,:,:,0])


    x2[:,:,:,1] = x2[:,:,:,0]
    x2[:,:,:,2] = x2[:,:,:,0]
    x2 -= .5
    x2 *= 2
    
    return x2


def generate_batches(files, idxt, params):
    nn = len(files)
    counter = 0
    cnt = 0
    
    while True:

        hdf5_file1 = h5py.File(files[counter], "r")
        print(hdf5_file1["labels"].shape)

        for cbatch in np.arange(0,idxt.shape[0],params.batch_size):
            idx = idxt[cbatch:(cbatch+params.batch_size)]
            idx.sort()

            x1 = (hdf5_file1["img"][idx,:,:,:])
            y1 = (hdf5_file1["labels"][idx,:2])

            x11 = create_patches(x1,params,idx.shape[0])

            yield (x11,y1)

        hdf5_file1.close()

        counter += 1
        if counter == nn:
            counter = 0 
    return

## Deep CNN classifier using keras


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, argparse
os.environ['KERAS_BACKEND'] = 'tensorflow'

import numpy as np

from keras.models import Model,load_model
from keras.layers import Dense,Activation
from keras.regularizers import l1

from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

from keras.optimizers import Adam
from keras_applications.resnet_v2 import ResNet50V2

import tensorflow as tf
import keras.backend as K

In [0]:
from sklearn.model_selection import StratifiedKFold

def run_CNN(params):

    f = 0
    gen_test = generate_batches(files=[params.input+"test.hdf5"], idxt=np.arange(11494), params=params)

    model,model_name,loadmdl = MyModel(params,f)

    print('sgd = Adam(lr=3e-3)')
    sgd = Adam(3e-3)

    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=["accuracy"])

    print("Compiled...") 

    nsamplesperfold = 1.*params.ntrans*params.npt/(params.batch_size)

    eval1 = model.evaluate_generator(gen_test, steps=11494,
                   verbose=1)
    print(eval1)

    return

In [0]:
def MyModel(params,f):

  
    model_name = params.models+str(f)+"epochs:002-val_acc:0.960.hdf5epochs:005-val_acc:0.999.hdf5"
    if not os.path.isfile(model_name ):
        print("Creating Model : ", model_name)
        model_base = ResNet50V2(include_top=False, weights='imagenet', input_shape=(params.psize[0], params.psize[1],params.nch), pooling='avg',backend=keras.backend, layers=keras.layers, models=keras.models, utils=keras.utils)

        x = model_base.get_layer('avg_pool').output
        dense = Dense(2,
                    kernel_initializer='he_normal',
                    bias_initializer='zeros',
                    kernel_regularizer=l1(0.01),
                    name='classifier')(x)
  
        prediction = Activation("softmax",name="softmax")(dense)

        model = Model(model_base.inputs, outputs=prediction, name='ResNet')

        loadmdl = 0
    else:
        print("Loading model: ", model_name)
        model_base = ResNet50V2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(params.psize[0], params.psize[1],params.nch), pooling='avg', classes=3)

        x = model_base.get_layer('avg_pool').output
        dense = Dense(2,
                    kernel_initializer='he_normal',
                    bias_initializer='zeros',
                    kernel_regularizer=l1(0.01),
                    name='classifier')(x)
  
        prediction = Activation("softmax",name="softmax")(dense)

        model = Model(model_base.inputs, outputs=prediction, name='ResNet')
      
        model.load_weights(model_name)


        loadmdl = 1


    return model,model_name,loadmdl




In [13]:
sys.argv = ['foo']

parser = argparse.ArgumentParser()

parser.add_argument('-input',
                    type=str,
                    default='/home/ubuntu/volume1/BrainHack/data/BrainHack_quality_',
                    help='txt file containing input data filenames')

parser.add_argument('-models', type=str, 
                    default='/home/ubuntu/volume1/BrainHack/models/model_BrainHack_CTMR_h5_',
                    help='models prefix directory')

parser.add_argument('-psize', type=int, default=[128,128],
                    help='patch size extracted from dataset')

parser.add_argument('-window', type=int, default=[224,224],
                    help='area of interest ')

parser.add_argument('-img_size', type=int, default=[224,224],
                    help='area of interest ')

parser.add_argument('-nfolds', type=int, default=1,
                    help='number of folds for training')

parser.add_argument('-nch', type=int, default=3,
                    help='number of input channels')

parser.add_argument('-npt', type=int, default=1,
                    help='the number of patches from slices')

parser.add_argument('-ntrans', type=int, default=5,
                    help='number of transformations ')

parser.add_argument('-batch_size', type=int, default=10,
                    help='size of batch')

params = parser.parse_args()

run_CNN(params)

Loading model:  /content/drive/My Drive/model_BrainHack_CTMR_h5_0epochs:002-val_acc:0.960.hdf5epochs:005-val_acc:0.999.hdf5
sgd = Adam(lr=3e-3)
Compiled...
(11494, 15)
 2289/11494 [====>.........................] - ETA: 8:27(11494, 15)
(11494, 15) 3440/11494 [=======>......................]
 6890/11494 [================>.............] - ETA: 4:06(11494, 15)
(11494, 15) 8040/11494 [===================>..........]
11494/11494 [==============================] - 612s 53ms/step
[0.018093021989381432, 0.9988684433396807]
